In [1]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
from torch.distributions import Uniform
# from torch.optim.lr_scheduler import ReduceLROnPlateau

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger

import torchmetrics

import numpy as np
import matplotlib.pyplot as plt

import wandb

import pandas as pd

import tensorflow as tf
from lbn_NAC import LBN

PATH_DATASETS = "."
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 500 if AVAIL_GPUS else 64
# BATCH_SIZE=1

In [2]:
pl.seed_everything(125)

Global seed set to 125


125

In [3]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [4]:
def mass(x):
    return torch.sqrt(x[...,0]**2 - x[...,1]**2 - x[...,2]**2 - x[...,3]**2)

In [5]:
def create_four_vectors(n, p_low=-100., p_high=100., m_low=0.1, m_high=50.):
    """
    Creates a numpy array with shape ``n + (4,)`` describing four-vectors of particles whose
    momentum components are uniformly distributed between *p_low* and *p_high*, and masses between
    *m_low* and *m_high*.
    """
    # create random four-vectors
    if not isinstance(n, tuple):
        n = (n,)
    vecs = np.random.uniform(p_low, p_high, n + (4,)).astype(np.float32)

    # the energy is also random and might be lower than the momentum,
    # so draw uniformly distributed masses, and compute and insert the energy
    m = np.abs(np.random.uniform(m_low, m_high, n))
    p = np.sqrt(np.sum(vecs[..., 1:]**2, axis=-1))
    E = (p**2 + m**2)**0.5
    vecs[..., 0] = E

    return vecs

In [6]:
def group_by_mass(p):
    m = mass(p)
    g1 = torch.where(m >= 25.)[0]
    g = torch.zeros(m.shape)
    g[g1] = 1.0
    return g.reshape(-1,1)

In [7]:
p = create_four_vectors(100)
p = p.reshape(-1,1,4)

In [8]:
lbn = LBN(10, boost_mode=LBN.PAIRS)
lbn.build(p.shape, features=["E", "pt", "eta", "phi", "m"])

2022-04-15 09:04:21.473510: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-15 09:04:21.474000: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-15 09:04:21.474179: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-15 09:04:21.474481: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [9]:
p.shape

(100, 1, 4)

In [10]:
lbn(p).shape

TensorShape([100, 50])

In [11]:
class ToyData(Dataset):
    def __init__(self, p_lbn, g, m):
        self.p_lbn = p_lbn
        self.g = g
        self.m = m
    
    def __len__(self):
        return self.p_lbn.shape[0]
    
    def __getitem__(self, idx):
        return self.p_lbn[idx,:], self.g[idx], self.m[idx]

## 1. LBN + DNN

In [12]:
class DNN_with_LBN(pl.LightningModule):
    def __init__(self, N=20000, hparams=None):
        super().__init__()
        
        self.N = N
        
        hidden_layer = hparams["hidden_layer"]
        hidden_depth = hparams["hidden_depth"]
        learning_rate = hparams["learning_rate"]
        batch_size = hparams["batch_size"]
        
        self.hidden_layer = hidden_layer
        self.hidden_depth = hidden_depth
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        
#         self.max_lr = hparams["max_lr"]
        self.epochs = hparams["epochs"]
        
        layers = [nn.Linear(50, hidden_layer), nn.ReLU(), nn.BatchNorm1d(hidden_layer)]
        for i in range(hidden_depth):
            layers.extend([
                nn.Linear(hidden_layer, hidden_layer),
                nn.ReLU(),
                nn.BatchNorm1d(hidden_layer)
            ])
        layers.append(nn.Linear(hidden_layer, 1))
        self.net = nn.Sequential(*layers)
        
        hparams = {
            "hidden_layer": hidden_layer,
            "hidden_depth": hidden_depth,
            "batch_size": batch_size,
        }
    
        self.save_hyperparameters(hparams)
        self.accuracy = torchmetrics.Accuracy()
        self.ds = None
        
    def forward(self, x):
        return self.net(x)
    
    def training_step(self, batch, batch_idx):
        p, g, _ = batch
        m = self(p)
        loss = F.binary_cross_entropy_with_logits(m, g)
        return loss
    
    def validation_step(self, batch, batch_idx):
        p, g, m_ans = batch
        m = self(p)
        loss = F.binary_cross_entropy_with_logits(m, g)
        acc = self.accuracy(
            torch.round(torch.sigmoid(m)),
            g.int()
        )
        with torch.no_grad():
            criterion = nn.MSELoss(reduction="mean")
            rmse = torch.sqrt(criterion(m + 25, m_ans) + 1e-6)
        
        self.log('val_loss', loss)
        self.log('val_acc', acc)
        self.log('rmse', rmse)
        
        return loss
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer
    
    def prepare_data(self):
        p = create_four_vectors(self.N)
        m = mass(torch.tensor(p))
        g = group_by_mass(torch.tensor(p))
        
        p = p.reshape(-1,1,4)
        lbn = LBN(10, boost_mode=LBN.PAIRS)
        lbn.build(p.shape, features=["E", "pt", "eta", "phi", "m"])
        p_lbn = lbn(p).numpy()
        
        self.W_hat = torch.tensor(lbn.W_hat.numpy())
        self.M_hat = torch.tensor(lbn.M_hat.numpy())
        self.ds = ToyData(p_lbn, g, m)
        
    def setup(self, stage=None):
        N_train = self.N // 10 * 7
        N_val = self.N - N_train
        if stage == "fit" or stage is None:
            self.ds_train, self.ds_val = random_split(self.ds, [N_train, N_val])
        if stage == "test" or stage is None:
            _, self.ds_test = random_split(self.ds, [N_train, N_val])
            
    def train_dataloader(self):
        return DataLoader(self.ds_train, batch_size=self.batch_size)
    
    def val_dataloader(self):
        return DataLoader(self.ds_val, batch_size=self.batch_size)
    
    def test_dataloader(self):
        return DataLoader(self.ds_test, batch_size=self.batch_size)

In [13]:
hparams = {
    "learning_rate": 1e-4,
    "batch_size": BATCH_SIZE,
    "epochs": 200,
    "hidden_layer": 64,
    "hidden_depth": 3,
}

model = DNN_with_LBN(
    hparams=hparams
)

wandb_logger = WandbLogger(
    project='LBN_Tutorial'
)

trainer = Trainer(
    logger=wandb_logger,
    max_epochs=hparams["epochs"],
    gpus=AVAIL_GPUS,
    enable_progress_bar=False,
    callbacks=[
        EarlyStopping(monitor="val_loss", patience=20, mode="min"),
    ]
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [14]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: Currently logged in as: axect (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.14 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade



  | Name     | Type       | Params
----------------------------------------
0 | net      | Sequential | 16.3 K
1 | accuracy | Accuracy   | 0     
----------------------------------------
16.3 K    Trainable params
0         Non-trainable params
16.3 K    Total params
0.065     Total estimated model params size (MB)
/home/xteca/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/xteca/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([500])) that is different to the input size (torch.Size([500, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  retur

In [15]:
wandb.finish()

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
rmse,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇██
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
val_acc,▁▂▂▂▂▃▃▃▃▃▃▄▄▅▅▆▆▆▇▇▇▇▇▇▇▇██████████████
val_loss,██▇▇▇▇▇▇▇▇▇▆▆▅▅▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,150
rmse,17.58071
trainer/global_step,4227
val_acc,0.9215
val_loss,0.22758


In [16]:
model.eval()
val_data = list(iter(model.val_dataloader()))

In [17]:
ps = val_data[0][0]
gs = val_data[0][1]
ms = val_data[0][2]
m_hats = model(ps)
g_hats = torch.round(torch.sigmoid(m_hats))
m_hats = m_hats + 25

ps = ps.detach().numpy()
gs = gs.detach().numpy()
ms = ms.detach().numpy()
m_hats = m_hats.detach().numpy()
g_hats = g_hats.detach().numpy()

In [18]:
dg = pd.DataFrame({
    "m": ms,
    "m_hat": m_hats[:,0],
    "g": gs[:,0],
    "g_hat": g_hats[:,0]
})

dg

,m,m_hat,g,g_hat
0,6.636046,6.354189,0.0,0.0
1,42.882839,33.568871,1.0,1.0
2,31.482122,27.915802,1.0,1.0
3,0.922801,13.494654,0.0,0.0
4,9.219436,14.974447,0.0,0.0
...,...,...,...,...
495,45.320107,36.525131,1.0,1.0
496,48.277569,35.954433,1.0,1.0
497,22.755516,25.992397,0.0,1.0
498,35.825844,32.032360,1.0,1.0


In [19]:
np.sqrt(np.sum((ms - m_hats[:,0])**2)) / len(ms)

0.29027395629882813